In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
DOCUMENT= """
The London trio are up for best UK act and best album, as
well as getting two nominations in the best song category. "We
got told like this morning ’Oh I think you’re nominated’", said
Dappy. "And I was like ’Oh yeah, which one?’ And now we’ve
got nominated for four awards. I mean, wow!"""

In [6]:
import re

DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()


In [7]:
!pip install transformers==4.26.1
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from summarizer import Summarizer

In [9]:
sm = Summarizer(model='bert-large-uncased')

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
result = sm(body=DOCUMENT, ratio=0.10)

In [43]:
result = '\n'.join(nltk.sent_tokenize(result))
print(result)

The London trio are up for best UK act and best album, as well as getting two nominations in the best song category. "
We got told like this morning ’Oh I think you’re nominated’", said Dappy. "


## **Abstractive Text Summarization** 

In [13]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
BART_PATH = 'facebook/bart-large-cnn'

In [14]:
bart_model = BartForConditionalGeneration.from_pretrained(BART_PATH, output_past=True)
bart_tokenizer = BartTokenizer.from_pretrained(BART_PATH, output_past=True)

In [15]:
def nest_sentences(document):

  nested = []
  sent = []
  length = 0
  for sentence in nltk.sent_tokenize(document):
    length += len(sentence)
    if length < 1024:
      sent.append(sentence)
    else:
      nested.append(sent)
      sent = []
      length = 0

  if sent:
    nested.append(sent)

  return nested

In [16]:
nested = nest_sentences(DOCUMENT)

In [17]:
nested[0]

['The London trio are up for best UK act and best album, as well as getting two nominations in the best song category.',
 '"We got told like this morning ’Oh I think you’re nominated’", said Dappy.',
 '"And I was like ’Oh yeah, which one?’ And now we’ve got nominated for four awards.',
 'I mean, wow!']

In [20]:
device = 'cuda'

In [21]:
input_tokenized = bart_tokenizer.encode(' '.join(nested[0]), truncation=True, return_tensors='pt')
input_tokenized = input_tokenized.to(device)
input_tokenized

tensor([[    0,   133,   928,  8566,    32,    62,    13,   275,   987,  1760,
             8,   275,  2642,     6,    25,   157,    25,   562,    80, 11097,
            11,     5,   275,  2214,  4120,     4,    22,   170,   300,   174,
           101,    42,   662,    44,    27,  7516,    38,   206,    47,    17,
            27,   241,  7076,    17,    27,  1297,    26,   211, 31953,     4,
            22,  2409,    38,    21,   101,    44,    27,  7516, 11380,     6,
            61,    65,   116,    17,    27,   178,   122,    52,    17,    27,
           548,   300,  7076,    13,   237,  4188,     4,    38,  1266,     6,
         26388,   328,     2]], device='cuda:0')

In [22]:
summary_ids = bart_model.to('cuda').generate(input_tokenized,
                                      length_penalty=3.0,
                                      min_length=30,
                                      max_length=100)
summary_ids

tensor([[    2,     0,   133,   928,  8566,    32,    62,    13,   275,   987,
          1760,     8,   275,  2642,     6,    25,   157,    25,   562,    80,
         11097,    11,     5,   275,  2214,  4120,     4,    22,   170,   300,
           174,   101,    42,   662,    44,    27,  7516,    38,   206,    47,
            17,    27,   241,  7076,    17,    27,  1297,    26,   211, 31953,
             4,     2]], device='cuda:0')

In [23]:
output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
output

['The London trio are up for best UK act and best album, as well as getting two nominations in the best song category. "We got told like this morning ’Oh I think you’re nominated’", said Dappy.']

In [24]:
nested[0]

['The London trio are up for best UK act and best album, as well as getting two nominations in the best song category.',
 '"We got told like this morning ’Oh I think you’re nominated’", said Dappy.',
 '"And I was like ’Oh yeah, which one?’ And now we’ve got nominated for four awards.',
 'I mean, wow!']

In [25]:
def generate_summary(nested_sentences):
  device = 'cuda'
  summaries = []
  for nested in nested_sentences:
    input_tokenized = bart_tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = bart_model.to('cuda').generate(input_tokenized,
                                      length_penalty=3.0,
                                      min_length=30,
                                      max_length=100)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [26]:
summ = generate_summary(nested)

In [27]:
summ

['The London trio are up for best UK act and best album, as well as getting two nominations in the best song category. "We got told like this morning ’Oh I think you’re nominated’", said Dappy.']

In [28]:
nested_summ = nest_sentences(' '.join(summ))
nested_summ

[['The London trio are up for best UK act and best album, as well as getting two nominations in the best song category.',
  '"We got told like this morning ’Oh I think you’re nominated’", said Dappy.']]

In [29]:
generate_summary(nested_summ)

['The London trio are up for best UK act and best album. They also have two nominations in the best song category. "We got told like this morning ’Oh I think you’re nominated’", said Dappy.']

### **Another Example**

In [30]:
DOCUMENT = """
Since late November, Scotland’s five mountain resorts have
attracted 373,782 customers. The ski season is estimated to
have attracted £37.5m into the local economy. With fresh snow
on the slopes, CairnGorm Mountain expects skiing during the
first weekend of June. Recent figures from Ski Scotland showed
that this season’s figures were better than the last bumper season
of 2000-2001.
"""

In [31]:
DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()

In [32]:
doc_nest = nest_sentences(DOCUMENT)
doc_nest

[['Since late November, Scotland’s five mountain resorts have attracted 373,782 customers.',
  'The ski season is estimated to have attracted £37.5m into the local economy.',
  'With fresh snow on the slopes, CairnGorm Mountain expects skiing during the first weekend of June.',
  'Recent figures from Ski Scotland showed that this season’s figures were better than the last bumper season of 2000-2001.']]

In [33]:
summ = generate_summary(doc_nest)
summ

['Since late November, Scotland’s five mountain resorts have attracted 373,782 customers. With fresh snow on the slopes, CairnGorm Mountain expects skiing during the first weekend of June.']

In [34]:
nest_summ = nest_sentences(' '.join(summ))
nest_summ

[['Since late November, Scotland’s five mountain resorts have attracted 373,782 customers.',
  'With fresh snow on the slopes, CairnGorm Mountain expects skiing during the first weekend of June.']]

**Text summary**

In [35]:
final_summ = generate_summary(nest_summ)
nest_sentences(' '.join(final_summ))

[['Since late November, Scotland’s five mountain resorts have attracted 373,782 customers.',
  'CairnGorm Mountain expects skiing during the first weekend of June.']]

## **Another one example** 

In [36]:
DOCUMENT = """
ChatGPT is a large language model (LLM), a machine-learning system that autonomously learns from data and can produce sophisticated and seemingly 
intelligent writing after training on a massive data set of text. It is the latest in a series of such models released by OpenAI, an AI company in 
San Francisco, California, and by other firms. ChatGPT has caused excitement and controversy because it is one of the first models that can convincingly 
converse with its users in English and other languages on a wide range of topics. It is free, easy to use and continues to learn.
This technology has far-reaching consequences for science and society. Researchers and others have already used ChatGPT and other large language models 
to write essays and talks, summarize literature, draft and improve papers, as well as identify research gaps and write computer code, including statistical analyses. 
Soon this technology will evolve to the point that it can design experiments, write and complete manuscripts, conduct peer review and support editorial decisions 
to accept or reject manuscripts.
Conversational AI is likely to revolutionize research practices and publishing, creating both opportunities and concerns. It might accelerate the innovation 
process, shorten time-to-publication and, by helping people to write fluently, make science more equitable and increase the diversity of scientific perspectives. 
However, it could also degrade the quality and transparency of research and fundamentally alter our autonomy as human researchers. ChatGPT and other LLMs produce 
text that is convincing, but often wrong, so their use can distort scientific facts and spread misinformation.
We think that the use of this technology is inevitable, therefore, banning it will not work. It is imperative that the research community engage in a debate 
about the implications of this potentially disruptive technology. Here, we outline five key issues and suggest where to start.
"""

In [37]:
DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()

In [38]:
doc_nest = nest_sentences(DOCUMENT)
doc_nest

[['ChatGPT is a large language model (LLM), a machine-learning system that autonomously learns from data and can produce sophisticated and seemingly intelligent writing after training on a massive data set of text.',
  'It is the latest in a series of such models released by OpenAI, an AI company in San Francisco, California, and by other firms.',
  'ChatGPT has caused excitement and controversy because it is one of the first models that can convincingly converse with its users in English and other languages on a wide range of topics.',
  'It is free, easy to use and continues to learn.',
  'This technology has far-reaching consequences for science and society.',
  'Researchers and others have already used ChatGPT and other large language models to write essays and talks, summarize literature, draft and improve papers, as well as identify research gaps and write computer code, including statistical analyses.'],
 ['Conversational AI is likely to revolutionize research practices and publ

In [39]:
summ = generate_summary(doc_nest)
summ

['ChatGPT is a machine-learning system that autonomously learns from data. It is the latest in a series of such models released by OpenAI, an AI company in San Francisco, California.',
 'ChatGPT and other LLMs produce text that is convincing, but often wrong. Their use can distort scientific facts and spread misinformation. We think that the use of this technology is inevitable, therefore, banning it will not work.']

In [40]:
nest_summ = nest_sentences(' '.join(summ))
nest_summ

[['ChatGPT is a machine-learning system that autonomously learns from data.',
  'It is the latest in a series of such models released by OpenAI, an AI company in San Francisco, California.',
  'ChatGPT and other LLMs produce text that is convincing, but often wrong.',
  'Their use can distort scientific facts and spread misinformation.',
  'We think that the use of this technology is inevitable, therefore, banning it will not work.']]

**Text summary**

In [44]:
final_summ = generate_summary(nest_summ)
nest_sentences(' '.join(final_summ))

[['ChatGPT is a machine-learning system that autonomously learns from data.',
  'It is the latest in a series of such models released by OpenAI, an AI company in San Francisco.']]